# Branch and Bound Algorithm

## Theory

- Branch and bound is an algorithm design paradigm which is generally used for solving combinatorial optimization problems. 
- These problems are typically exponential in terms of time complexity and may require exploring all possible permutations in worst case.
- The Branch and Bound Algorithm technique solves these problems relatively quickly.


- The Backtracking Solution can be optimized if we know a bound on best possible solution subtree rooted with every node. 
- If the best in subtree is worse than current best, we can simply ignore this node and its subtrees. 
- So we compute bound (best solution) for every node and compare the bound with current best solution before exploring the node

In [5]:
import threading as th
from copy import deepcopy


class Node:
    def __init__(self, parent=None, state=[]):
        self.parent = parent
        self.generator_lock = th.Lock()
        self.generator = self._child_gen()
        self.state = state

    def _child_gen(self):
        for i in range(1, 4):
            state = deepcopy(self.state) + [i]
            yield Node(self, state)

    def next_child(self):
        with self.generator_lock:
            return next(self.generator, None)

    def is_leaf(self):
        return len(self.state) >= 10

    def __repr__(self):
        return '<Node state="{}">'.format(self.state)


class Worker:
    def __init__(self, id, searcher):
        self.searcher = searcher  # type: Searcher
        self.id = id

    def __call__(self):
        print("start worker: {}".format(self.id))
        while not self.searcher.is_end():
            self._run()
        print("end worker: {}".format(self.id))

    def _run(self):
        node = self.searcher.get_last_node()

        if node is None:
            return

        if node.is_leaf():
            self.searcher.remove_node(node)
            self.searcher.add_result(node)
            return

        bounds = self.searcher.get_bounds()
        if not self.satisfy_bounds(node, bounds):
            self.searcher.remove_node(node)
            return

        child = node.next_child()
        if child is None:
            self.searcher.remove_node(node)
        else:
            self.searcher.add_node(child)

    def satisfy_bounds(self, node, bound):
        return True


class Searcher:
    def __init__(self):
        self.root_node = Node()
        self.nodes = [self.root_node]  # TODO: priority queue
        self.nodes_lock = th.Lock()

        self._is_end = False
        self.workers = [ Worker(i, self) for i in range(8) ]

        self.results = set()
        self.results_lock = th.Lock()

        self.bounds = [None, None]
        self.bounds_lock = th.Lock()
        self.threads = []

    def run(self):
        self.threads = [
            th.Thread(target=w, name="thread:{}".format(idx))
            for idx, w in enumerate(self.workers)
            ]
        for t in self.threads:
            t.start()
        for t in self.threads:
            t.join()

    def get_last_node(self):
        with self.nodes_lock:
            if self.nodes:
                return self.nodes[-1]
            else:
                self._is_end = True
                return None

    def add_node(self, node):
        with self.nodes_lock:
            self.nodes.append(node)

    def remove_node(self, node):
        with self.nodes_lock:
            if node in self.nodes:
                self.nodes.remove(node)

    def is_end(self):
        return self._is_end

    def check_end(self):
        with self.nodes_lock:
            self._is_end = len(self.nodes) == 0

    def add_result(self, node):
        with self.results_lock:
            self.results.add(node)

    def get_bounds(self):
        with self.bounds_lock:
            return deepcopy(self.bounds)


def main():
    s = Searcher()
    s.run()
    print(len(s.results))
    assert len(s.results) == 3 ** 10


if __name__ == '__main__':
    main()


start worker: 0
start worker: 1
start worker: 2
start worker: 3
start worker: 4
start worker: 5
start worker: 6
start worker: 7
end worker: 4end worker: 5end worker: 0
end worker: 6
end worker: 7


end worker: 1end worker: 3
end worker: 2

59049


In [ ]:
graph = {
    'S': {'A': 3, 'B': 2},
    'A': {'C': 4, 'D': 1, 'S': 3},
    'B': {'E': 3, 'F': 1, 'S': 2},
    'C': {'A': 4},
    'D': {'A': 1},
    'E': {'B': 3, 'H': 5},
    'F': {'B': 1, 'I': 2, 'G': 3},
    'G': {'F': 3},
    'I': {'F': 2},
    'H': {'E': 5}
}

In [2]:
graph = {
    'S': { 'A':4, 'B': 3},
    'A': { 'S':4, 'E':15, 'B': 3, 'D':8},
    'B': { 'S':4, 'A':3, 'D':6, 'C':4}, 
    'C': { 'B': 4, 'D': 4, 'G': 7},
    'D': { 'A': 8, 'G': 2, 'E': 2, 'B': 6, 'C': 4},
    'E': { 'A': 15, 'D': 2},
    'G': { 'C':7, 'D': 2}
}

In [ ]:
def branch_bound():
    